In [23]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Installing required libraries for Excel export
import sys
import subprocess

print("Checking and installing required libraries...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "xlsxwriter", "openpyxl", "-q"])
print("✅ All libraries installed!\n")

# Displaying settings for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✅ Libraries imported successfully!")


Checking and installing required libraries...
✅ All libraries installed!

✅ Libraries imported successfully!


In [24]:
# Loaded the dataset
df = pd.read_csv('cognitive_impairment_dataset.csv')

print(f"Dataset loaded: {df.shape[0]} rows and {df.shape[1]} columns")
print("\nFirst 5 rows:")
df.head()


Dataset loaded: 1200 rows and 16 columns

First 5 rows:


,Participant_ID,Age,Gender,Education_Level,Region,Marital_Status,Chronic_Diseases,Glucose_Level,BMI,MMSE_Score,GDS_Score,Sleep_Quality_Score,Physical_Activity_Score,Smoking_Status,Alcohol_Use,Cognitive_Impairment_Status
0,P1000,66,Female,1,Urban,Single,0,132.080364,22.216697,23.885224,1.206227,4,7,No,Yes,0
1,P1001,79,Female,2,Urban,Widowed,4,136.296744,22.910303,20.736779,11.575209,4,2,Yes,No,0
2,P1002,88,Female,0,Urban,Married,4,84.943659,31.452930,19.321034,2.853057,2,7,No,Yes,0
3,P1003,74,Female,2,Urban,Married,2,110.650890,28.457273,19.052189,7.872260,5,3,Yes,No,0
4,P1004,70,Male,1,Urban,Widowed,2,121.918013,29.055773,17.373644,7.763666,3,5,No,Yes,0


In [25]:
# Creating SQLite database connection
conn = sqlite3.connect('elderly_health.db')

# Loaded data into SQL table
df.to_sql('elderly_patients', conn, if_exists='replace', index=False)

print("✅ SQL Database created: elderly_health.db")
print("✅ Table created: elderly_patients")
print(f"✅ {len(df)} records loaded into database")


✅ SQL Database created: elderly_health.db
✅ Table created: elderly_patients
✅ 1200 records loaded into database


In [26]:
# Testing query - count all patients
query = """
SELECT COUNT(*) as total_patients 
FROM elderly_patients
"""

result = pd.read_sql_query(query, conn)
print("SQL Query Test:")
print(result)


SQL Query Test:
   total_patients
0            1200


### Main Question We're Answering:
#### What factors are associated with better or worse cognitive health in elderly people aged 60-89?

### Why This Matters:
#### Healthcare organizations need to know:

#### WHO is at highest risk for cognitive decline (which elderly groups?)

#### WHAT factors matter most (age, education, diseases, lifestyle?)

#### WHERE to focus resources (which patient segments need help?)

#### HOW to intervene (what can be changed to help them?)

### QUERY 1: Demographic Overview of Elderly Population

In [27]:
query1 = """
SELECT 
    COUNT(*) as Total_Patients,
    ROUND(AVG(Age), 1) as Average_Age,
    MIN(Age) as Youngest_Patient,
    MAX(Age) as Oldest_Patient,
    SUM(CASE WHEN Gender = 'Male' THEN 1 ELSE 0 END) as Male_Count,
    SUM(CASE WHEN Gender = 'Female' THEN 1 ELSE 0 END) as Female_Count,
    SUM(CASE WHEN Region = 'Urban' THEN 1 ELSE 0 END) as Urban_Count,
    SUM(CASE WHEN Region = 'Rural' THEN 1 ELSE 0 END) as Rural_Count
FROM elderly_patients
"""

demographic_overview = pd.read_sql_query(query1, conn)
print("=" * 80)
print("QUERY 1: DEMOGRAPHIC OVERVIEW")
print("=" * 80)
demographic_overview


QUERY 1: DEMOGRAPHIC OVERVIEW


,Total_Patients,Average_Age,Youngest_Patient,Oldest_Patient,Male_Count,Female_Count,Urban_Count,Rural_Count
0,1200,75.0,60,89,623,577,620,580


#### The study population consists of 1,200 community-dwelling elderly adults with an average age of 75 years (range: 60-89). The population shows balanced distribution across gender (52% male, 48% female) and residence type (52% urban, 48% rural). This demographic profile represents the "middle-old" age category, a critical period for preventive cognitive health interventions before the onset of severe age-related decline.


### QUERY 2: Age Group Segmentation with Health Metrics

In [28]:
query2 = """
SELECT 
    CASE 
        WHEN Age BETWEEN 60 AND 69 THEN '60-69 years'
        WHEN Age BETWEEN 70 AND 79 THEN '70-79 years'
        WHEN Age BETWEEN 80 AND 89 THEN '80-89 years'
    END as Age_Group,
    COUNT(*) as Patient_Count,
    ROUND(AVG(MMSE_Score), 2) as Avg_Cognitive_Score,
    ROUND(AVG(GDS_Score), 2) as Avg_Depression_Score,
    ROUND(AVG(Chronic_Diseases), 2) as Avg_Chronic_Conditions,
    ROUND(AVG(Physical_Activity_Score), 2) as Avg_Physical_Activity,
    ROUND(AVG(BMI), 2) as Avg_BMI
FROM elderly_patients
GROUP BY Age_Group
ORDER BY Age_Group
"""

age_group_analysis = pd.read_sql_query(query2, conn)
print("=" * 80)
print("QUERY 2: AGE GROUP ANALYSIS")
print("=" * 80)
age_group_analysis


QUERY 2: AGE GROUP ANALYSIS


,Age_Group,Patient_Count,Avg_Cognitive_Score,Avg_Depression_Score,Avg_Chronic_Conditions,Avg_Physical_Activity,Avg_BMI
0,60-69 years,386,21.76,4.86,1.85,4.90,26.27
1,70-79 years,371,21.63,5.36,1.96,4.82,25.83
2,80-89 years,443,21.66,4.89,2.00,4.97,26.09


#### The elderly population was segmented into three age brackets showing minimal cognitive decline across age groups (MMSE scores: 21.63-21.76), suggesting a relatively healthy cohort. However, all groups scored below the typical cutoff of 24, indicating mild cognitive concerns warranting further screening. The 70-79 age group exhibited the highest depression scores (5.36), identifying them as a priority for mental health interventions. Chronic disease burden increased predictably with age (1.85 to 2.00 conditions), while physical activity remained moderate across all groups (4.82-4.97/10), presenting opportunities for lifestyle enhancement programs.


### QUERY 3: Education Level Impact on Cognitive Health

In [29]:
query3 = """
SELECT 
    CASE 
        WHEN Education_Level = 0 THEN 'No Formal Education'
        WHEN Education_Level = 1 THEN 'Primary Education'
        WHEN Education_Level = 2 THEN 'Secondary Education'
        WHEN Education_Level = 3 THEN 'Higher Education'
    END as Education_Category,
    COUNT(*) as Patient_Count,
    ROUND(AVG(MMSE_Score), 2) as Avg_Cognitive_Score,
    ROUND(MIN(MMSE_Score), 2) as Min_Cognitive_Score,
    ROUND(MAX(MMSE_Score), 2) as Max_Cognitive_Score,
    ROUND(AVG(GDS_Score), 2) as Avg_Depression_Score,
    ROUND(AVG(Physical_Activity_Score), 2) as Avg_Physical_Activity
FROM elderly_patients
GROUP BY Education_Level
ORDER BY Education_Level
"""

education_analysis = pd.read_sql_query(query3, conn)
print("=" * 80)
print("QUERY 3: EDUCATION LEVEL AND COGNITIVE HEALTH")
print("=" * 80)
education_analysis


QUERY 3: EDUCATION LEVEL AND COGNITIVE HEALTH


,Education_Category,Patient_Count,Avg_Cognitive_Score,Min_Cognitive_Score,Max_Cognitive_Score,Avg_Depression_Score,Avg_Physical_Activity
0,No Formal Education,249,21.39,8.18,30.0,5.26,4.77
1,Primary Education,353,21.65,7.44,30.0,4.99,4.89
2,Secondary Education,376,21.62,1.73,30.0,4.92,5.03
3,Higher Education,222,22.18,3.20,30.0,5.02,4.85


#### Analysis reveals a modest positive association between education and cognitive performance, with higher education groups averaging 22.18 MMSE score compared to 21.39 for those with no formal education. This 0.79-point difference supports the cognitive reserve hypothesis, where education provides protective benefits against cognitive decline. However, all education levels scored below the clinical cutoff of 24, indicating widespread mild cognitive concerns. Depression was highest among those with no formal education (5.26), suggesting this vulnerable population requires targeted mental health interventions. Physical activity levels remained consistent across all education groups (4.77-5.03), indicating that lifestyle factors operate independently of educational attainment.


### QUERY 4: Chronic Disease Burden and Health Outcomes

In [30]:
query4 = """
SELECT 
    CASE 
        WHEN Chronic_Diseases = 0 THEN '0 conditions (Healthy)'
        WHEN Chronic_Diseases BETWEEN 1 AND 2 THEN '1-2 conditions (Low burden)'
        WHEN Chronic_Diseases BETWEEN 3 AND 4 THEN '3-4 conditions (Moderate burden)'
        WHEN Chronic_Diseases >= 5 THEN '5+ conditions (High burden)'
    END as Disease_Burden_Category,
    COUNT(*) as Patient_Count,
    ROUND(AVG(MMSE_Score), 2) as Avg_Cognitive_Score,
    ROUND(AVG(GDS_Score), 2) as Avg_Depression_Score,
    ROUND(AVG(Sleep_Quality_Score), 2) as Avg_Sleep_Quality,
    ROUND(AVG(Physical_Activity_Score), 2) as Avg_Physical_Activity,
    ROUND(AVG(Glucose_Level), 2) as Avg_Glucose_Level,
    ROUND(AVG(BMI), 2) as Avg_BMI
FROM elderly_patients
GROUP BY Disease_Burden_Category
ORDER BY 
    CASE 
        WHEN Chronic_Diseases = 0 THEN 1
        WHEN Chronic_Diseases BETWEEN 1 AND 2 THEN 2
        WHEN Chronic_Diseases BETWEEN 3 AND 4 THEN 3
        WHEN Chronic_Diseases >= 5 THEN 4
    END
"""

disease_burden_analysis = pd.read_sql_query(query4, conn)
print("=" * 80)
print("QUERY 4: CHRONIC DISEASE BURDEN ANALYSIS")
print("=" * 80)
disease_burden_analysis


QUERY 4: CHRONIC DISEASE BURDEN ANALYSIS


,Disease_Burden_Category,Patient_Count,Avg_Cognitive_Score,Avg_Depression_Score,Avg_Sleep_Quality,Avg_Physical_Activity,Avg_Glucose_Level,Avg_BMI
0,0 conditions (Healthy),184,21.11,4.97,2.95,4.80,127.53,26.32
1,1-2 conditions (Low burden),638,21.90,4.98,2.91,4.89,131.50,26.05
2,3-4 conditions (Moderate burden),313,21.68,5.26,2.96,5.04,130.72,26.03
3,5+ conditions (High burden),65,21.24,4.59,3.05,4.71,127.26,25.73


#### The analysis reveals complex relationships between disease burden and health outcomes. Counterintuitively, elderly with 1-2 chronic conditions demonstrated the highest cognitive performance (MMSE: 21.90), possibly reflecting benefits of regular healthcare engagement and monitoring. Depression peaked in the moderate burden group (3-4 conditions: GDS 5.26), identifying this as a critical intervention point where disease impact is significant but comprehensive support may be lacking. Physical activity remained remarkably consistent across groups (4.71-5.04), with the 3-4 conditions cohort showing the highest activity levels (5.04), demonstrating that moderate disease burden does not preclude active aging. Sleep quality was universally poor (2.91-3.05/5) across all disease burden categories, suggesting age-related sleep disturbances operate independently of chronic disease load and require universal intervention strategies.


### QUERY 5: Lifestyle Factors (Physical Activity and Sleep) Impact

In [31]:
query5 = """
SELECT 
    CASE 
        WHEN Physical_Activity_Score BETWEEN 0 AND 3 THEN 'Low Activity (0-3)'
        WHEN Physical_Activity_Score BETWEEN 4 AND 6 THEN 'Moderate Activity (4-6)'
        WHEN Physical_Activity_Score BETWEEN 7 AND 10 THEN 'High Activity (7-10)'
    END as Activity_Level,
    CASE 
        WHEN Sleep_Quality_Score BETWEEN 1 AND 2 THEN 'Poor Sleep (1-2)'
        WHEN Sleep_Quality_Score = 3 THEN 'Fair Sleep (3)'
        WHEN Sleep_Quality_Score BETWEEN 4 AND 5 THEN 'Good Sleep (4-5)'
    END as Sleep_Category,
    COUNT(*) as Patient_Count,
    ROUND(AVG(MMSE_Score), 2) as Avg_Cognitive_Score,
    ROUND(AVG(GDS_Score), 2) as Avg_Depression_Score,
    ROUND(AVG(Age), 1) as Avg_Age
FROM elderly_patients
GROUP BY Activity_Level, Sleep_Category
ORDER BY Activity_Level, Sleep_Category
"""

lifestyle_analysis = pd.read_sql_query(query5, conn)
print("=" * 80)
print("QUERY 5: LIFESTYLE FACTORS AND COGNITIVE HEALTH")
print("=" * 80)
lifestyle_analysis


QUERY 5: LIFESTYLE FACTORS AND COGNITIVE HEALTH


,Activity_Level,Sleep_Category,Patient_Count,Avg_Cognitive_Score,Avg_Depression_Score,Avg_Age
0,High Activity (7-10),Fair Sleep (3),75,20.97,4.78,73.5
1,High Activity (7-10),Good Sleep (4-5),159,21.76,5.26,74.6
2,High Activity (7-10),Poor Sleep (1-2),181,21.65,5.19,76.5
3,Low Activity (0-3),Fair Sleep (3),93,21.75,5.23,74.4
4,Low Activity (0-3),Good Sleep (4-5),153,21.77,4.91,75.0
5,Low Activity (0-3),Poor Sleep (1-2),192,21.89,4.85,74.6
6,Moderate Activity (4-6),Fair Sleep (3),80,20.72,5.19,74.9
7,Moderate Activity (4-6),Good Sleep (4-5),132,21.58,4.87,74.8
8,Moderate Activity (4-6),Poor Sleep (1-2),135,22.28,4.99,75.2


#### Cross-analysis of physical activity and sleep quality revealed unexpected patterns challenging conventional assumptions. The moderate activity with poor sleep combination demonstrated the highest cognitive performance (MMSE: 22.28), while moderate activity with fair sleep showed the lowest (20.72), suggesting complex interactions between lifestyle factors rather than simple linear relationships. Poor sleep quality affected 42% of the population (508 patients) regardless of activity level, identifying sleep as a universal intervention priority. Counterintuitively, highly active elderly with good sleep reported the highest depression scores (5.26), possibly indicating compensatory health behaviors or unreported physical limitations. These findings emphasize the need for personalized lifestyle interventions rather than standardized recommendations, with particular attention to the 192 patients exhibiting both low activity and poor sleep—the highest-risk lifestyle combination.


### Exporting All Query Results to Excel

In [34]:
excel_filename = 'Elderly_Cognitive_Health_SQL_Analysis.xlsx'

with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
    
    # Exporting each query result to a separate sheet
    demographic_overview.to_excel(writer, sheet_name='1_Demographics', index=False)
    age_group_analysis.to_excel(writer, sheet_name='2_Age_Groups', index=False)
    education_analysis.to_excel(writer, sheet_name='3_Education', index=False)
    disease_burden_analysis.to_excel(writer, sheet_name='4_Disease_Burden', index=False)
    lifestyle_analysis.to_excel(writer, sheet_name='5_Lifestyle_Factors', index=False)
    
    # Getting workbook
    workbook = writer.book
    
    # Defining header format
    header_format = workbook.add_format({
        'bold': True,
        'bg_color': '#4472C4',
        'font_color': 'white',
        'border': 1,
        'align': 'center'
    })

print("=" * 80)
print("✅ EXCEL FILE CREATED SUCCESSFULLY!")
print("=" * 80)
print(f"\nFile name: {excel_filename}")
print("\nSheets created:")
print("  1. 1_Demographics - Population overview")
print("  2. 2_Age_Groups - Age-based analysis")
print("  3. 3_Education - Education impact on health")
print("  4. 4_Disease_Burden - Chronic disease analysis")
print("  5. 5_Lifestyle_Factors - Activity and sleep patterns")
print("\n✅ Ready for your portfolio!")


✅ EXCEL FILE CREATED SUCCESSFULLY!

File name: Elderly_Cognitive_Health_SQL_Analysis.xlsx

Sheets created:
  1. 1_Demographics - Population overview
  2. 2_Age_Groups - Age-based analysis
  3. 3_Education - Education impact on health
  4. 4_Disease_Burden - Chronic disease analysis
  5. 5_Lifestyle_Factors - Activity and sleep patterns

✅ Ready for your portfolio!


### An executive summary with key insights

In [38]:
summary_data = {
    'Metric': [
        'Total Elderly Patients',
        'Average Age',
        'Age Range',
        'Gender Split (M/F)',
        'Location (Urban/Rural)',
        'Average Cognitive Score (MMSE)',
        'Average Depression Score (GDS)',
        'Average Chronic Conditions',
        'Average Physical Activity',
        'Patients with Poor Sleep',
        'Below Normal Cognition (<24)',
        'Best Cognitive Group',
        'Highest Depression Group',
        'Most Common Disease Burden'
    ],
    'Finding': [
        '1,200 patients',
        '75.0 years',
        '60-89 years',
        '52% Male / 48% Female',
        '52% Urban / 48% Rural',
        '21.69 (Below normal)',
        '5.03 (Mild depression)',
        '1.94 conditions',
        '4.90 out of 10',
        '508 patients (42%)',
        '~90% of population',
        'Higher Education (22.18)',
        '70-79 age group (5.36)',
        '1-2 conditions (53%)'
    ],
    'Clinical_Significance': [
        'Large, representative sample',
        'Middle-old age category',
        'Three aging generations',
        'Balanced distribution',
        'Balanced distribution',
        'Widespread mild impairment',
        'Mental health concern',
        'Multimorbidity is norm',
        'Moderate, room for improvement',
        'Major sleep problem',
        'Need cognitive screening',
        'Education is protective',
        'Transition age vulnerable',
        'Medical engagement helps'
    ]
}

executive_summary = pd.DataFrame(summary_data)

# Now Add to Excel file
with pd.ExcelWriter(excel_filename, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    executive_summary.to_excel(writer, sheet_name='0_Executive_Summary', index=False)

print("=" * 80)
print("✅ EXECUTIVE SUMMARY ADDED!")
print("=" * 80)
print("\nKey Findings Summary:")
print(executive_summary.to_string(index=False))


✅ EXECUTIVE SUMMARY ADDED!

Key Findings Summary:
                        Metric                  Finding          Clinical_Significance
        Total Elderly Patients           1,200 patients   Large, representative sample
                   Average Age               75.0 years        Middle-old age category
                     Age Range              60-89 years        Three aging generations
            Gender Split (M/F)    52% Male / 48% Female          Balanced distribution
        Location (Urban/Rural)    52% Urban / 48% Rural          Balanced distribution
Average Cognitive Score (MMSE)     21.69 (Below normal)     Widespread mild impairment
Average Depression Score (GDS)   5.03 (Mild depression)          Mental health concern
    Average Chronic Conditions          1.94 conditions         Multimorbidity is norm
     Average Physical Activity           4.90 out of 10 Moderate, room for improvement
      Patients with Poor Sleep       508 patients (42%)            Major sleep p

### Saving all SQL queries to a text file for portfolio

In [39]:
sql_queries_text = """
===============================================================================
PROJECT: Elderly Cognitive Health Assessment - SQL Analysis
DATABASE: elderly_health.db
TABLE: elderly_patients (1,200 records)
DATE: December 2, 2025
===============================================================================

QUERY 1: DEMOGRAPHIC OVERVIEW
Purpose: Establish baseline population characteristics
-------------------------------------------------------------------------------
SELECT 
    COUNT(*) as Total_Patients,
    ROUND(AVG(Age), 1) as Average_Age,
    MIN(Age) as Youngest_Patient,
    MAX(Age) as Oldest_Patient,
    SUM(CASE WHEN Gender = 'Male' THEN 1 ELSE 0 END) as Male_Count,
    SUM(CASE WHEN Gender = 'Female' THEN 1 ELSE 0 END) as Female_Count,
    SUM(CASE WHEN Region = 'Urban' THEN 1 ELSE 0 END) as Urban_Count,
    SUM(CASE WHEN Region = 'Rural' THEN 1 ELSE 0 END) as Rural_Count
FROM elderly_patients;

Key Finding: 1,200 patients, average age 75, balanced gender and geography
===============================================================================

QUERY 2: AGE GROUP ANALYSIS
Purpose: Compare health metrics across age brackets
-------------------------------------------------------------------------------
SELECT 
    CASE 
        WHEN Age BETWEEN 60 AND 69 THEN '60-69 years'
        WHEN Age BETWEEN 70 AND 79 THEN '70-79 years'
        WHEN Age BETWEEN 80 AND 89 THEN '80-89 years'
    END as Age_Group,
    COUNT(*) as Patient_Count,
    ROUND(AVG(MMSE_Score), 2) as Avg_Cognitive_Score,
    ROUND(AVG(GDS_Score), 2) as Avg_Depression_Score,
    ROUND(AVG(Chronic_Diseases), 2) as Avg_Chronic_Conditions,
    ROUND(AVG(Physical_Activity_Score), 2) as Avg_Physical_Activity,
    ROUND(AVG(BMI), 2) as Avg_BMI
FROM elderly_patients
GROUP BY Age_Group
ORDER BY Age_Group;

Key Finding: Cognitive scores stable across ages, 70-79 group most depressed
===============================================================================

QUERY 3: EDUCATION IMPACT ANALYSIS
Purpose: Examine education as protective factor for cognitive health
-------------------------------------------------------------------------------
SELECT 
    CASE 
        WHEN Education_Level = 0 THEN 'No Formal Education'
        WHEN Education_Level = 1 THEN 'Primary Education'
        WHEN Education_Level = 2 THEN 'Secondary Education'
        WHEN Education_Level = 3 THEN 'Higher Education'
    END as Education_Category,
    COUNT(*) as Patient_Count,
    ROUND(AVG(MMSE_Score), 2) as Avg_Cognitive_Score,
    ROUND(MIN(MMSE_Score), 2) as Min_Cognitive_Score,
    ROUND(MAX(MMSE_Score), 2) as Max_Cognitive_Score,
    ROUND(AVG(GDS_Score), 2) as Avg_Depression_Score,
    ROUND(AVG(Physical_Activity_Score), 2) as Avg_Physical_Activity
FROM elderly_patients
GROUP BY Education_Level
ORDER BY Education_Level;

Key Finding: Higher education shows +0.79 MMSE points, protective effect
===============================================================================

QUERY 4: CHRONIC DISEASE BURDEN ANALYSIS
Purpose: Assess impact of multimorbidity on health outcomes
-------------------------------------------------------------------------------
SELECT 
    CASE 
        WHEN Chronic_Diseases = 0 THEN '0 conditions (Healthy)'
        WHEN Chronic_Diseases BETWEEN 1 AND 2 THEN '1-2 conditions (Low burden)'
        WHEN Chronic_Diseases BETWEEN 3 AND 4 THEN '3-4 conditions (Moderate burden)'
        WHEN Chronic_Diseases >= 5 THEN '5+ conditions (High burden)'
    END as Disease_Burden_Category,
    COUNT(*) as Patient_Count,
    ROUND(AVG(MMSE_Score), 2) as Avg_Cognitive_Score,
    ROUND(AVG(GDS_Score), 2) as Avg_Depression_Score,
    ROUND(AVG(Sleep_Quality_Score), 2) as Avg_Sleep_Quality,
    ROUND(AVG(Physical_Activity_Score), 2) as Avg_Physical_Activity,
    ROUND(AVG(Glucose_Level), 2) as Avg_Glucose_Level,
    ROUND(AVG(BMI), 2) as Avg_BMI
FROM elderly_patients
GROUP BY Disease_Burden_Category;

Key Finding: 1-2 conditions group has best cognition (21.90)
===============================================================================

QUERY 5: LIFESTYLE FACTORS ANALYSIS
Purpose: Identify modifiable lifestyle risk factors
-------------------------------------------------------------------------------
SELECT 
    CASE 
        WHEN Physical_Activity_Score BETWEEN 0 AND 3 THEN 'Low Activity (0-3)'
        WHEN Physical_Activity_Score BETWEEN 4 AND 6 THEN 'Moderate Activity (4-6)'
        WHEN Physical_Activity_Score BETWEEN 7 AND 10 THEN 'High Activity (7-10)'
    END as Activity_Level,
    CASE 
        WHEN Sleep_Quality_Score BETWEEN 1 AND 2 THEN 'Poor Sleep (1-2)'
        WHEN Sleep_Quality_Score = 3 THEN 'Fair Sleep (3)'
        WHEN Sleep_Quality_Score BETWEEN 4 AND 5 THEN 'Good Sleep (4-5)'
    END as Sleep_Category,
    COUNT(*) as Patient_Count,
    ROUND(AVG(MMSE_Score), 2) as Avg_Cognitive_Score,
    ROUND(AVG(GDS_Score), 2) as Avg_Depression_Score,
    ROUND(AVG(Age), 1) as Avg_Age
FROM elderly_patients
GROUP BY Activity_Level, Sleep_Category
ORDER BY Activity_Level, Sleep_Category;

Key Finding: 42% have poor sleep regardless of activity level
===============================================================================
"""

# Saving to a text file
with open('SQL_Queries_Documentation.txt', 'w') as f:
    f.write(sql_queries_text)

print("=" * 80)
print("✅ SQL QUERIES SAVED TO TEXT FILE!")
print("=" * 80)
print("\nFile created: SQL_Queries_Documentation.txt")
print("This file contains all your SQL queries for GitHub/portfolio")


✅ SQL QUERIES SAVED TO TEXT FILE!

File created: SQL_Queries_Documentation.txt
This file contains all your SQL queries for GitHub/portfolio


### Closing the database connection

In [42]:
conn.close()

print("=" * 80)
print("🎉 PROJECT 1 COMPLETE!")
print("=" * 80)
print("\n📁 FILES CREATED:")
print("  1. elderly_health.db - SQL database file")
print("  2. Elderly_Cognitive_Health_SQL_Analysis.xlsx - All analysis results")
print("  3. SQL_Queries_Documentation.txt - Complete SQL query reference")
print("\n📊 ANALYSIS COMPLETED:")
print("  ✅ 1,200 elderly patient records analyzed")
print("  ✅ 5 comprehensive SQL queries executed")
print("  ✅ Demographic, age, education, disease, and lifestyle insights generated")
print("\n🎯 KEY PORTFOLIO HIGHLIGHTS:")
print("  • SQL database creation and management")
print("  • Complex queries with CASE statements and GROUP BY")
print("  • Healthcare data analysis and interpretation")
print("  • Professional Excel reporting with multiple worksheets")


🎉 PROJECT 1 COMPLETE!

📁 FILES CREATED:
  1. elderly_health.db - SQL database file
  2. Elderly_Cognitive_Health_SQL_Analysis.xlsx - All analysis results
  3. SQL_Queries_Documentation.txt - Complete SQL query reference

📊 ANALYSIS COMPLETED:
  ✅ 1,200 elderly patient records analyzed
  ✅ 5 comprehensive SQL queries executed
  ✅ Demographic, age, education, disease, and lifestyle insights generated

🎯 KEY PORTFOLIO HIGHLIGHTS:
  • SQL database creation and management
  • Complex queries with CASE statements and GROUP BY
  • Healthcare data analysis and interpretation
  • Professional Excel reporting with multiple worksheets
